<a href="https://colab.research.google.com/github/rishipython/FakeNewsDetection/blob/main/FakeNewsDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set-Up

In [ ]:
import pandas as pd
import numpy as np
import io
import os
import re
import shutil
import string
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Dropout, Layer
from tensorflow.keras.layers import Embedding, Input, GlobalAveragePooling1D, Dense
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential, Model
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

# Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/fake_or_real_news.csv')

In [ ]:
df

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
...,...,...,...,...
6330,4490,State Department says it can't find emails fro...,The State Department told the Republican Natio...,REAL
6331,8062,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,FAKE
6332,8622,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,FAKE
6333,4021,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",REAL


In [ ]:
data = df.copy()
data.drop("Unnamed: 0",axis=1,inplace=True)

In [ ]:
data

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
...,...,...,...
6330,State Department says it can't find emails fro...,The State Department told the Republican Natio...,REAL
6331,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,FAKE
6332,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,FAKE
6333,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",REAL


In [ ]:
from sklearn.preprocessing import LabelEncoder
lb=LabelEncoder()
data['label']=lb.fit_transform(data['label'])
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

cv=CountVectorizer(max_features=5000)
tf=TfidfVectorizer()
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
data

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",0
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,0
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,1
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",0
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,1
...,...,...,...
6330,State Department says it can't find emails fro...,The State Department told the Republican Natio...,1
6331,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,0
6332,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,0
6333,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",1


In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
from tqdm import tqdm

final=[]
temp=[]
full=[]
for i in tqdm(range(len(data))):
    tt=data['text'][i].lower()
    tt=re.sub('[^a-zA-Z]'," ",tt)
    temp=tt.split()
    final=[lemmatizer.lemmatize(i) for i in temp if not i in stopwords.words('english')]
    full.append(" ".join(final))

100%|██████████| 6335/6335 [12:38<00:00,  8.35it/s]


In [ ]:
cv.transform([full[0]]).toarray().shape

(1, 5000)

In [ ]:
pickle.dump(cv, open('cv.pkl', 'bw'))

In [ ]:
X=cv.fit_transform(full).toarray()

In [ ]:
X.shape

(6335, 5000)

In [ ]:
y = data['label']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
mb=MultinomialNB()

mb.fit(X_train,y_train)

MultinomialNB()

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=10, tol=None)
sgd.fit(X_train, y_train)

SGDClassifier(alpha=0.001, max_iter=10, random_state=42, tol=None)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred=mb.predict(X_test)
accuracy_score(y_pred,y_test)

0.866571018651363

In [ ]:
np.save('/content/drive/MyDrive/fakenewsdetection_X_train.npy', X_train)
np.save('/content/drive/MyDrive/fakenewsdetection_X_test.npy', X_test)
np.save('/content/drive/MyDrive/fakenewsdetection_y_train.npy', y_train)
np.save('/content/drive/MyDrive/fakenewsdetection_y_test.npy', y_test)

In [ ]:
import pickle
pickle.dump(mb, open("/content/drive/MyDrive/fakenewsdetection_MultinomialNB.pkl", 'bw'))
pickle.dump(sgd, open("/content/drive/MyDrive/fakenewsdetection_SGDClassifier.pkl", 'bw'))

In [ ]:
import json
model_param = {}
model_param['coef'] = list(sgd.coef_[0])
model_param['intercept'] = sgd.intercept_.tolist()
model_param['classes'] = sgd.classes_.tolist()

In [ ]:
json_txt = json.dumps(model_param, indent=4)
with open('/content/drive/MyDrive/FakeNewsDetection/model_param.txt', 'w') as file:
  file.write(json_txt)

In [ ]:
from sklearn.linear_model import SGDClassifier

dict_ = json.load(open('model_param.txt', 'r'))
sgd_blank = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=10, tol=None)
sgd_blank.coef_ = np.array([dict_['coef']])
sgd_blank.intercept_ = np.array(dict_['intercept'])
sgd_blank.classes_ = np.array(dict_['classes'])

In [ ]:
accuracy_score(sgd_blank.predict(X_test), y_test)

0.9024390243902439

In [ ]:
sgd.predict(np.array([X_test[0]]))

array([0])

In [ ]:
X_test[0]

array([0, 0, 0, ..., 0, 1, 0])

In [ ]:
y_test[1357]

0

# Run Model!

In [ ]:
def text_to_vect(text):
    tt=text.lower()
    tt=re.sub('[^a-zA-Z]'," ",tt)
    temp=tt.split()
    final=[lemmatizer.lemmatize(i) for i in temp if not i in stopwords.words('english')]
    return cv.transform([" ".join(final)]).toarray()

In [ ]:
dict = {0:"Fake 😥", 1:"Real 😍"}

while True:
  text = input("Enter text: ")
  vect = text_to_vect(text)
  print()
  print(dict[sgd.predict(vect)[0]])

Enter text: This has turned into the whiplash election, and itâ€™s virtually impossible to keep up with the head-snapping revelations.  Even the most diligent journalist can get vertigo trying to investigate and evaluate each disclosure, which damages Hillary Clinton or Donald Trump and spawns dire predictions that their campaign is toast, only to fade in a news cycle or two.  Some developments that in a â€œnormalâ€ election would badly wound a presidential candidate get meager exposure in the rest of the media because they are overshadowed by other mega-stories. No one has the bandwidth to do it all.  This is more than a mere campaign; its craziness has become the beating heart of American culture, debated in every cubicle and coffee shop and at every level of the increasingly toxic stew of social media.  Just look at the last few weeks:  Democrats and many members of the media have taken up weapons and stormed a place called Comey Island.  The FBI director shook up the race by revea